# How to get a call transcript from Call Analytics

Prerequisites: 
* Access to an account with Chime SDK Call Analytics enabled with Transcribe Call Analytics and data lake enabled.
* Access to Athena for SQL queries on the data lake

For the purpose of this example, we will be using a demo account and a demo transaction ID (you will have to replace with your relevant details) to extract the transcript for a particular call on 5/28/23.

In [20]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
import warnings
warnings.filterwarnings('ignore')
import sys
!{sys.executable} -m pip install PyAthena

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [ ]:
#For example: s3_location = 's3://abcde-demo-us-east-1/demo/' 
print("Enter S3 location")
s3_location = input()

In [ ]:
#For example: region = 'us-east-1'
print("Enter region name")
region = input()

In [23]:
from pyathena import connect
import pandas as pd
conn = connect(s3_staging_dir=s3_location,
               region_name= region)

Let's say we're interested in the call with transaction ID = '12345678-1234-1234-1234-123456789abc' (you will need to replace this with an actual transaction ID). 

In [ ]:
##For example: transactionid = '12345678-1234-1234-1234-123456789abc' 
print("Enter transaction ID")
transactionid = input()

The following SQL query fetches the transcript for the call.

In [27]:
df = pd.read_sql("""SELECT time, 
                    utteranceevent.participantrole,
                    utteranceevent.transcript
                    FROM AwsDataCatalog.amazon_chime_sdk_database.transcribe_call_analytics
                    where json_extract_scalar(metadata,'$.transactionId') = '{}'
                    and utteranceevent.ispartial = false
                    ORDER BY time ASC
                    """.format(transactionid), conn)

df

,time,participantrole,transcript
0,2023-05-28T02:28:39.837511,AGENT,Thank you for calling. Mechanic Michelle Seattle's only all female auto shopping. This is Maria. How can I help you today?
1,2023-05-28T02:28:45.855283,CUSTOMER,"Hi, Maria. Uh, thank you for asking, Uh, I actually have several issues with my car."
2,2023-05-28T02:28:50.803107,CUSTOMER,"Um, to begin with. I see a check engine light on the dashboard, and I'm not sure what"
3,2023-05-28T02:28:53.029513,CUSTOMER,"That indicates in addition,"
4,2023-05-28T02:28:59.007071,CUSTOMER,My dashboard frequently shows warnings straight to sensor malfunctions of all kinds.
5,2023-05-28T02:29:04.049181,CUSTOMER,"Um I do find it difficult to start the car, sometimes, especially in winter and summer."
6,2023-05-28T02:29:05.297370,CUSTOMER,Sometimes I hear
7,2023-05-28T02:29:08.797294,CUSTOMER,Click right before it starts up. I'm not sure what that is.
8,2023-05-28T02:29:13.511857,CUSTOMER,"Uh, my engine often makes buttering noises."
9,2023-05-28T02:29:18.055307,CUSTOMER,"Um, my steering wheel shakes when I drive the vehicle."


In [28]:
transcript = ""
for index, row in df.iterrows():
    transcript += row['participantrole'] + ": " + row['transcript'] + "\n"
    
print(transcript)

AGENT: Thank you for calling. Mechanic Michelle Seattle's only all female auto shopping. This is Maria. How can I help you today?
CUSTOMER: Hi, Maria. Uh, thank you for asking, Uh, I actually have several issues with my car.
CUSTOMER: Um, to begin with. I see a check engine light on the dashboard, and I'm not sure what
CUSTOMER: That indicates in addition,
CUSTOMER: My dashboard frequently shows warnings straight to sensor malfunctions of all kinds.
CUSTOMER: Um I do find it difficult to start the car, sometimes, especially in winter and summer.
CUSTOMER: Sometimes I hear
CUSTOMER: Click right before it starts up. I'm not sure what that is.
CUSTOMER: Uh, my engine often makes buttering noises.
CUSTOMER: Um, my steering wheel shakes when I drive the vehicle.
AGENT: Hmm.
CUSTOMER: Sometimes, when I turned the steering wheel, right, the car swings left and vice versa.
CUSTOMER: The brakes makes creaking or grinding noises, sometimes
CUSTOMER: My car is consuming too much oil and constantl